In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights

import numpy as np
import pandas as pd
from fastparquet import write

import os
import glob as glob

import cv2
from PIL import Image


%matplotlib inline

In [5]:
class FinalLayer(nn.Module):
    """Modified last layer for resnet50 for your dataset"""
    def __init__(self):
        super(FinalLayer, self).__init__()
        self.fc = nn.Linear(2048, 12)  # Assuming you have 12 output classes
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        out = self.sigmoid(out)
        return out

def modified_resnet50():
    # Load pretrained resnet50 with a modified last fully connected layer
    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    model.fc = FinalLayer()
    return model


def model_load():
    """ Loads pre-trained weights to set the protest detection model """
    # Load the modified ResNet-50 model
    model = modified_resnet50()

    # Load the protest prediction model (dictionary)
    model_checkpoint = torch.load('../../protest-detection-violence-estimation/model_best.pth.tar')
    model.load_state_dict(model_checkpoint['state_dict'])
    model.eval()

    if torch.cuda.is_available():
        model = model.to('cuda')
    
    return model

In [16]:
import datasets

ImportError: cannot import name 'get_data_patterns' from 'datasets.data_files' (/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/yolo5_transf_learn_env/lib/python3.9/site-packages/datasets/data_files.py)

In [6]:
test_swe_path = '/zpool/beast-mirror/labour-movements-mobilisation-via-visual-means/youtube_video_frames/sweden/6Fackforbund/8fX3nxZe79U'


In [8]:
data_transforms = {
    'predict': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor()
    ])}

dataset = {'predict' : datasets.ImageFolder(test_swe_path, data_transforms['predict'])}
dataloader = {'predict': torch.utils.data.DataLoader(dataset['predict'], batch_size = 1, shuffle=False, num_workers=4)}


model = model_load()

outputs = list()
for inputs, labels in dataloader['predict']:
    inputs = inputs.to(device)
    output = model(inputs)
    #output = output.to(device)
    #index = output.data.numpy().argmax()
    print(output)
    
    

"""def image_transform(image):
    # Define image transformations
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match the model's input size
        transforms.ToTensor(),           # Convert to tensor
    ])
    return preprocess(image)
    
def protest_inference(model, path_to_img:str):
    # Load your input image
    image = Image.open(path_to_img)

    # Preprocess the image
    input_tensor = image_transform(image)

    # Add a batch dimension (1 image)
    input_tensor = input_tensor.unsqueeze(0)

    # Move all to cuda 
    if torch.cuda.is_available():
        input_tensor = input_tensor.to('cuda')

    # Perform inference
    with torch.no_grad():
        output = model(input_tensor)

    output = output.to('cpu')

    # Convert the output tensor to list
    output_list = output[0].tolist()
    
    return output_list"""


NameError: name 'datasets' is not defined